In [1]:
import pandas as pd
from matplotlib import pyplot as plt

In [3]:
tr = pd.read_pickle('../data/train.pkl')
tr_log = pd.read_pickle('../data/train_log.pkl')

In [5]:
tr.head()

,object_id,ra,decl,gal_l,gal_b,ddf,hostgal_specz,hostgal_photoz,hostgal_photoz_err,distmod,mwebv,target
0,615,349.046051,-61.943836,320.796530,-51.753706,1,0.0000,0.0000,0.0000,NaN,0.017,92
1,713,53.085938,-27.784405,223.525509,-54.460748,1,1.8181,1.6267,0.2552,45.4063,0.007,88
2,730,33.574219,-6.579593,170.455585,-61.548219,1,0.2320,0.2262,0.0157,40.2561,0.021,42
3,745,0.189873,-45.586655,328.254458,-68.969298,1,0.3037,0.2813,1.1523,40.7951,0.007,90
4,1124,352.711273,-63.823658,316.922299,-51.059403,1,0.1934,0.2415,0.0176,40.4166,0.024,90


In [6]:
tr.tail()

,object_id,ra,decl,gal_l,gal_b,ddf,hostgal_specz,hostgal_photoz,hostgal_photoz_err,distmod,mwebv,target
7843,130739978,26.718750,-14.940303,172.342697,-72.255675,0,0.0000,0.0000,0.0000,NaN,0.013,65
7844,130755807,120.101349,-62.696659,275.742955,-16.509746,0,0.1725,2.5606,1.1146,46.6108,0.136,90
7845,130762946,203.108109,-55.682144,308.728904,6.727511,0,0.0000,0.0000,0.0000,NaN,0.430,16
7846,130772921,79.101562,-35.501846,239.172243,-33.827844,0,0.0000,0.0000,0.0000,NaN,0.034,65
7847,130779836,301.992188,-17.426323,25.102988,-24.511101,0,0.0000,0.0000,0.0000,NaN,0.091,6


In [4]:
tr_log.head(20)

,object_id,mjd,passband,flux,flux_err,detected
0,615,59750.421875,2,-544.810303,3.622952,1
1,615,59750.429688,1,-816.434326,5.553370,1
2,615,59750.437500,3,-471.385529,3.801213,1
3,615,59750.445312,4,-388.984985,11.395031,1
4,615,59752.406250,2,-681.858887,4.041204,1
5,615,59752.414062,1,-1061.457031,6.472994,1
6,615,59752.421875,3,-524.954590,3.552751,1
7,615,59752.433594,4,-393.480225,3.599346,1
8,615,59752.445312,5,-355.886780,10.421921,1
9,615,59767.296875,2,-548.013550,3.462291,1


In [5]:
df = tr_log[tr_log.object_id==615]

In [7]:
df['date'] = df.mjd.astype(int)

/home/Kazuki/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [13]:
df

,object_id,mjd,passband,flux,flux_err,detected,date
0,615,59750.4229,2,-544.810303,3.622952,1,59750
1,615,59750.4306,1,-816.434326,5.553370,1,59750
2,615,59750.4383,3,-471.385529,3.801213,1,59750
3,615,59750.4450,4,-388.984985,11.395031,1,59750
4,615,59752.4070,2,-681.858887,4.041204,1,59752
5,615,59752.4147,1,-1061.457031,6.472994,1,59752
6,615,59752.4224,3,-524.954590,3.552751,1,59752
7,615,59752.4334,4,-393.480225,3.599346,1,59752
8,615,59752.4435,5,-355.886780,10.421921,1,59752
9,615,59767.2968,2,-548.013550,3.462291,1,59767


In [20]:
tmp = pd.pivot_table(df, index=['date'], columns=['passband'], values=['flux'])

In [21]:
tmp.index %= 365

In [22]:
tmp

flux                                                   \
passband           0            1           2           3           4   
date                                                                    
255              NaN  -816.434326 -544.810303 -471.385529 -388.984985   
257              NaN -1061.457031 -681.858887 -524.954590 -393.480225   
272              NaN  -815.188599 -548.013550 -475.516052 -405.663818   
275              NaN  -820.042786 -554.903198 -477.004730 -400.270386   
284              NaN  -921.002502 -630.523682 -518.533997 -422.184509   
287              NaN  -449.095612 -280.039520 -316.704865 -332.885437   
302              NaN    35.511822  391.399231  330.623901  360.397858   
305              NaN   129.541901  168.739899   30.120724  -60.942333   
312              NaN  -420.796417 -256.660980 -298.936859 -311.977783   
315              NaN  -527.020325 -342.819763 -363.282532 -348.628662   
318              NaN -1100.440063 -678.045715 -506.687408 -304.049713   
324         6.878784          NaN         NaN         NaN         NaN   
325        39.364853          NaN         NaN         NaN         NaN   
326       -10.422381          NaN         NaN         NaN         NaN   
327       -65.485130          NaN         NaN         NaN         NaN   
328      -113.349159          NaN         NaN         NaN         NaN   
340              NaN  -178.149399  -54.949490 -140.818436 -200.294128   
344              NaN  -953.883728 -639.035950 -518.293274 -418.723907   
347              NaN -1003.971497 -502.215332 -233.167755  111.507675   
356       -68.502457          NaN         NaN         NaN         NaN   
359              NaN   217.894211  459.452667  361.023438  374.446442   
362              NaN   646.523193  599.812195  354.961365  293.879608   
4                NaN  -942.167908 -637.105347 -524.586548 -414.447723   
7                NaN  -910.677734 -332.763123  -62.065010  202.288223   
10               NaN   659.486694  604.344543  373.986511  322.604034   
13               NaN   -98.796974    4.656033  -93.732880 -165.793457   
14        -97.353195          NaN         NaN         NaN         NaN   
15        -97.523880          NaN         NaN         NaN         NaN   
16       -108.672577          NaN         NaN         NaN         NaN   
17       -116.913223          NaN         NaN         NaN         NaN   
...              ...          ...         ...         ...         ...   
330        49.886921          NaN         NaN         NaN         NaN   
331         9.075453          NaN         NaN         NaN         NaN   
332       -30.764908          NaN         NaN         NaN         NaN   
333      -101.419899          NaN         NaN         NaN         NaN   
334      -110.688477          NaN         NaN         NaN         NaN   
335      -114.774445          NaN         NaN         NaN         NaN   
342              NaN  -972.201111 -447.681580 -176.163651  140.860611   
349              NaN  -812.792908 -143.843872   86.606873  257.570221   
352              NaN  -963.216980 -425.988464 -148.178238  161.872543   
355              NaN   655.284058  586.178345  445.737061  361.595764   
357       -51.614189          NaN         NaN         NaN         NaN   
358        20.364273          NaN         NaN         NaN         NaN   
359       -24.682575          NaN         NaN         NaN         NaN   
360       -63.546600          NaN         NaN         NaN         NaN   
361      -101.819290          NaN         NaN         NaN         NaN   
362      -110.978699          NaN         NaN         NaN         NaN   
363      -113.588432          NaN         NaN         NaN         NaN   
3                NaN   205.029755  226.696259   73.384720  -19.212976   
6                NaN  -388.231476 -224.917938 -274.108429 -292.558990   
13               NaN    70.494507  404.391388  338.994537  362.888550   
16               NaN   457.502197  422.610779  205.937546  123.048210   
1

In [ ]:
tr_log['date'] = tr_log.mjd.astype(int)
pd.pivot_table(tr_log, index=['object_id', 'date'], columns=['passband'], values=['flux'])